In [1]:
%matplotlib inline

import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import tensorflow as tf

In [2]:
X_y = pickle.load(open("all_image_array_train.pickle", "rb"))

In [3]:
X_y_test = pickle.load(open("all_image_array_test.pickle", "rb"))

In [4]:
X_y.shape

(10010, 785)

In [5]:
X_y[1:10,0]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [6]:
X_y[1:10,0][:,None]

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]])

In [7]:
enc = OneHotEncoder()
enc.fit(X_y[:,0][:,None])
y = np.asarray(enc.transform(X_y[:,0][:,None]).toarray())
y_test = np.asarray(enc.transform(X_y_test[:,0][:,None]).toarray())

In [8]:
y.shape

(10010, 10)

In [9]:
y_test.shape

(10009, 10)

In [10]:
y

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [11]:
X = X_y[:,1:785]
X_y_test = X_y_test[:,1:785]

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
graph = tf.Graph()
with graph.as_default():
    X_train_tf = tf.cast(tf.constant(X_train), tf.float32)
    y_train_tf = tf.cast(tf.constant(y_train), tf.float32)
    
    X_valid_tf = tf.cast(tf.constant(X_valid), tf.float32)
    
    X_test_tf = tf.cast(tf.constant(X_y_test), tf.float32)

    norm_dist_weights = tf.truncated_normal([X.shape[1], y.shape[1]])

    norm_dist_weights_tf = tf.Variable(norm_dist_weights)

    norm_dist_weights

    y.shape[1]

    biases_tf = tf.Variable(tf.zeros([y.shape[1]]))

    X_train.shape

    logits = tf.matmul(X_train_tf, norm_dist_weights_tf) + biases_tf
    tf_loss_alldata_perclass = tf.nn.softmax_cross_entropy_with_logits(logits, y_train_tf)
    print (tf_loss_alldata_perclass)
    loss = tf.reduce_mean(tf_loss_alldata_perclass)
    print(loss)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    y_train_prediction = tf.nn.softmax(logits)
    
    y_valid_prediction = tf.nn.softmax(
    tf.matmul(X_valid_tf,norm_dist_weights_tf) + biases_tf
    )

    y_test_prediction = tf.nn.softmax(
    tf.matmul(X_test_tf,norm_dist_weights_tf) + biases_tf
    )
    
    

Tensor("SoftmaxCrossEntropyWithLogits:0", shape=(6706,), dtype=float32)


In [14]:
X_train_tf

<tf.Tensor 'Cast:0' shape=(6706, 784) dtype=float32>

In [15]:
norm_dist_weights

<tf.Tensor 'truncated_normal:0' shape=(784, 10) dtype=float32>

In [16]:
logits

<tf.Tensor 'add:0' shape=(6706, 10) dtype=float32>

In [17]:
y_train_tf

<tf.Tensor 'Cast_1:0' shape=(6706, 10) dtype=float32>

In [18]:
num_steps = 801

def accuracy(predictions, label):
    return(100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(label, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ("Initialized")
    
    for step in range(num_steps):
        _, curr_loss, predictions = session.run([optimizer, loss, y_train_prediction])
        # below code is required for display purpose only
        
        if(step % 100 == 0):
            print('loss at step %d: %.4f' % (step, curr_loss))
            print('Training accuracy: %.2f%%' % accuracy(predictions, y_train))
            print('Validation accuracy: %.2f%%' % accuracy(y_valid_prediction.eval(), y_valid))
            print('Testing accuracy: %.2f%%' % accuracy(y_test_prediction.eval(), y_test))
            

Initialized
loss at step 0: 9463.0752
Training accuracy: 10.50%
Validation accuracy: 28.60%
Testing accuracy: 31.20%
loss at step 100: 62310.1523
Training accuracy: 73.26%
Validation accuracy: 72.06%
Testing accuracy: 79.67%
loss at step 200: 34646.9609
Training accuracy: 78.47%
Validation accuracy: 73.61%
Testing accuracy: 81.14%
loss at step 300: 54471.3594
Training accuracy: 70.76%
Validation accuracy: 68.77%
Testing accuracy: 75.40%
loss at step 400: 79131.2500
Training accuracy: 60.50%
Validation accuracy: 67.55%
Testing accuracy: 74.12%
loss at step 500: 47812.9531
Training accuracy: 73.11%
Validation accuracy: 70.97%
Testing accuracy: 78.13%
loss at step 600: 33268.1055
Training accuracy: 79.51%
Validation accuracy: 74.18%
Testing accuracy: 81.23%
loss at step 700: 46324.4922
Training accuracy: 74.80%
Validation accuracy: 72.15%
Testing accuracy: 80.00%
loss at step 800: 42465.0078
Training accuracy: 76.96%
Validation accuracy: 72.03%
Testing accuracy: 78.70%


In [19]:
#train_prediction = tf.nn.softmax(logits)
#valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases) 
#test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [20]:
graph_scd = tf.Graph()
with graph_scd.as_default():
    tf_X_train_scd = tf.placeholder(dtype=tf.float32, shape=X_train.shape)
    tf_y_train_scd = tf.placeholder(dtype=tf.float32, shape = y_train.shape)
    
    tf_weights_scd = tf.Variable(tf.truncated_normal(shape=[X_train.shape[1], y_train.shape[1]], dtype=tf.float32))
    tf_biases_scd = tf.Variable(tf.zeros(shape=y_train.shape[1]))
    
    logits = tf.matmul(a=tf_X_train_scd, b=tf_weights_scd) + tf_biases_scd
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_y_train_scd))    
    tf_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(loss)

In [ ]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    